# Designing a structure from the bottom up: r0b0t

**r0b0t** is a small program for interacting with Large Language Models you can find [here](https://github.com/lamg/r0b0t). This article shows how it was restructured by finding complex operations that could be contained and exposed through simpler interfaces.

## The [Stream](https://github.com/lamg/r0b0t/tree/master/Lib/Stream) module

A module is a group of elements whose interactions can be contained, abstracted and hidden under a small interface compared to the one that would be the result exposing their full domain.

The modules defined below follow that principle

Stream exposes the following interface

```fsharp
type GetProvider = unit -> AsyncSeq<string option>
type StopInsert = {insertWord: string -> unit; stop: unit -> unit}

val main: GetProvider -> StopInsert -> unit
```

The purpose of this module is to consume a `None` terminated sequence of strings, `AsyncSeq<string option>`, produced by an LLM, and send it to a GUI, hidden in the implementation of `StopInsert`. The choice of this module as starting point comes from the observation that this is one of the places were the most complex operations happen, namely, the coordination between the consumption of asynchronous data and its display on a GUI.

This coordination between asynchrounous stream of strings and GUI seems unavoidable for this project, therefore it's one of its atomic and complex components.

By focusing first in such complex interactions I try to create abstractions that will effectively contain concerns that otherwise could leak to other project components. That way experimentation and development can be done inside modules without outer interference. This doesn't imply the components are or should be reusable, because still their interfaces are quite particular to this project.

## The [GetProviderImpl](https://github.com/lamg/r0b0t/blob/master/Lib/GetProviderImpl.fs) module

The above module relies on `GetProvider` and `StopInsert`. We can focus now on implementing them. Let's start with a module for `GetProvider`:

```fsharp
type Conf =
  { active: Active
    providers: Map<Provider, ProviderImpl> }

val initConf: ProviderModule list -> Provider -> Conf
val getProvider: (unit -> Conf) -> (unit -> Prompt) : Stream.Types.GetProvider 
```

You can observe instead of `Conf` and `Prompt`, `getProvider` receives `unit -> Conf` and `unit -> Prompt`. The reason for this is the configuration and the prompt are values stored in mutable variables, in the User Interface.

## The [ProviderModule](https://github.com/lamg/r0b0t/tree/master/Lib/ProviderModuleImpl) implementation

In the above module, `initConf`, relies on `ProviderModule`, which is the interface that hides the interaction with GitHub Copilot, OpenAI's API, or any other that could be added in the future. You can find implementations for them under the directory `ProviderModuleImpl`.

Their exposed interface is the following

```fsharp
val providerModule: GetProviderImpl.ProviderModule
```

## [GUI](https://github.com/lamg/r0b0t/blob/master/Lib/GUI.fs) module: initialization and mutable state

With the above modules implemented there's nothing left that to create the context where the configuration is used and mutated by the GUI. This allows us to call `initConf`, `getProvider`, and finally `Stream.main` to get answers from LLMs. `Stream.main` also relies on `StopInsert`, but its implementation turned out to be so tied to the `GUI` module that it didn't deserve a separate one.